This notebook was written on colab. It requires CUDA and CUDNN. This notebook was used to train YOLOV3 on the Helmet dataset using pjreddie's darknet repository.  

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Clone darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15457, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 15457 (delta 4), reused 21 (delta 2), pack-reused 15424
Receiving objects: 100% (15457/15457), 14.12 MiB | 17.96 MiB/s, done.
Resolving deltas: 100% (10367/10367), done.


## Make the makefile changes 

In [ ]:
%cd /content/darknet/

!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile

#In src/detector.c change the weights save freq(line 394)

/content/darknet


## Make darknet build changes

In [ ]:
%cd /content/darknet

%pycat Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
%pycat Makefile

#Comment out the compute 30 arch line in MAKE file before building
#In examples/detection.c change the weights save freq(line 138)

## Build darknet and check build

In [ ]:
%cd /content/darknet
!make
!./darknet

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## Download model from drive

In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection', '/content/darknet/Yolo_builds/Helmet_detection')

'/content/darknet/Yolo_builds/Helmet_detection'

## Unzip the data

In [ ]:
%cd /content/darknet/Yolo_builds/Helmet_detection
!unzip Data_set/Images_and_annotations.zip -d Data_set

Streaming output truncated to the last 5000 lines.
  inflating: Data_set/Images_and_annotations/hard_hat_workers4499.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4476.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4690.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4761.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4449.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4404.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4468.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4663.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4581.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4526.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers473.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4395.txt  
  inflating: Data_set/Images_and_annotations/hard_hat_workers4493.txt  
  inflating: D

## Make the neccessary .conf and .data model file changes now

## Convert the file formats for linux

In [ ]:
%cd /content/darknet/Yolo_builds/Helmet_detection

!sudo apt install dos2unix
!dos2unix train_data.txt
!dos2unix valid_data.txt
!dos2unix Helmet_detection.data
!dos2unix class_names.txt
!dos2unix yolov3-voc.cfg

"""
classes= 3
backup = /content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save
names = Yolo_builds/Helmet_detection/class_names.txt
valid  = Yolo_builds/Helmet_detection/valid_data.txt
train  = Yolo_builds/Helmet_detection/train_data.txt
"""

/content/darknet/Yolo_builds/Helmet_detection
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 2s (233 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: f

'\nclasses= 3\nbackup = /content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save\nnames = Yolo_builds/Helmet_detection/class_names.txt\nvalid  = Yolo_builds/Helmet_detection/valid_data.txt\ntrain  = Yolo_builds/Helmet_detection/train_data.txt\n'

## Train

In [ ]:
%cd /content/darknet
#!./darknet detector train Yolo_builds/Helmet_detection/Helmet_detection.data Yolo_builds/Helmet_detection/yolov3-voc.cfg /content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save/yolov3-voc_4900.weights

!./darknet detector train Yolo_builds/Helmet_detection/Helmet_detection.data Yolo_builds/Helmet_detection/yolov4-tiny.cfg /content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save/yolov4-tiny_6500.weights -dont_show

/content/darknet
 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov4-tiny
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 16, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	   